In [94]:
import numpy as np
import datasets
from scipy.optimize import minimize
from scipy.special import expit
from sklearn.metrics import log_loss
from softmax import softmax, log_softmax
import theano
from theano import tensor as T
import tensorflow as tf

In [112]:
X, y = datasets.htwt()
Y = datasets.one_hot(y)
N, D = X.shape
N, C = Y.shape

In [152]:
mu = lambda X, w: expit(X.dot(w[1:]) + w[0])
loss = lambda w: log_loss(y, mu(X, w))
params = [0] * (D + 1)
w = minimize(loss, params).x
yhat = mu(X, w) > 0.5
print '%0.3f' % np.mean(yhat != y)

0.119


In [191]:
mu = lambda X, W, b: softmax(X.dot(W) + b)

loss = lambda params: log_loss(y, mu(X, *decode(params)))

def grad(params):
    Z = mu(X, *decode(params)) - Y
    dW = sum([np.kron(z, X[i]) for i, z in enumerate(Z)])
    dW = np.array(dW.reshape((C, C))).T
    db = np.sum(Z, axis = 0)
    return np.array(list(dW.ravel()) + list(db.ravel()))

def hess(params):
    Z = mu(X, *decode(params))
    o = lambda x: np.outer(x, x)
    dW = sum([np.kron(np.diag(z) - o(z), o(X[i])) for i, z in enumerate(Z)])
    db = sum([np.diag(z) - o(z) for i, z in enumerate(Z)])
    return np.array([0] * 36).reshape((6, 6))
    return np.array(list(dW.ravel()) + list(db.ravel()))

params = [0] * (D * (C - 1) + C)
decode = lambda params: (np.array((list(params[:D * (C - 1)]) + [0] * C)).reshape((D, C)), params[-C:])

params = [0] * (D * C + C)
decode = lambda params: (np.array((list(params[:D * C]))).reshape((D, C)), params[-C:])

# params = minimize(loss, params).x
params = minimize(loss, params, method = 'Newton-CG', jac = grad, hess = hess).x
yhat = np.argmax(mu(X, *decode(params)), axis = 1)
print '%0.3f' % np.mean(yhat != y)

0.348


In [120]:
decode = lambda params: (params[:D * C].reshape((D, C)), params[-C:])
z = T.dvector()
mu = lambda X, W, b: T.nnet.softmax(T.dot(X, W) + b)
loss = T.sum(T.nnet.categorical_crossentropy(mu(X, *decode(z)), Y))
grad = theano.function([z], T.grad(loss, z))
hess = theano.function([z], T.hessian(loss, z))    
loss = theano.function([z], loss)

params = [0] * (D * C + C)
decode = lambda params: (np.array((list(params[:D * C]))).reshape((D, C)), params[-C:])

# params = minimize(loss, params, method = 'Newton-CG', jac = grad, hess = hess).x
params = minimize(loss, params).x

decode = lambda params: (np.array((list(params[:D * C]))).reshape((D, C)), params[-C:])
mu = lambda X, W, b: softmax(X.dot(W) + b)
yhat = np.argmax(mu(X, *decode(params)), axis = 1)
print '%0.3f' % np.mean(yhat != y)

0.119


In [119]:
W = theano.shared(np.zeros((D, C)))
b = theano.shared(np.zeros(C))
tx = T.dmatrix()
ty = T.dmatrix()

loss = T.sum(T.nnet.categorical_crossentropy(T.nnet.softmax(T.dot(tx, W) + b), ty))
learning_rate = 0.01
optimizer = theano.function([tx, ty], updates =
    [[p, p - T.grad(loss, p) * learning_rate] for p in [W, b]]
)

optimizer(X, Y)
W, b = W.get_value(), b.get_value()
yhat = np.argmax(softmax(X.dot(W) + b), axis = 1)
print '%0.3f' % np.mean(yhat != y)

0.348


In [118]:
W = tf.Variable(tf.zeros((D, C)))
b = tf.Variable(tf.zeros(C))

loss = tf.nn.softmax_cross_entropy_with_logits(tf.matmul(X, W) + b, Y)
optimizer = tf.train.AdamOptimizer().minimize(loss)

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    sess.run(optimizer)
    W, b = W.eval(), b.eval()

yhat = np.argmax(softmax(X.dot(W) + b), axis = 1)
print '%0.3f' % np.mean(yhat != y)

0.348
